## Global Black-box Hyperparameter Optimisation Survey  

Global black-box optimisation is the problem of minimising an objective function that is entirely unknown or intractable, over a configuration space, in our case of machine learning, finding the best hyperparameters of a given algorithm that returns the best performance on a validation set. These hyperparameters range from the value of k in KNN, C, gamme and the kernel in a SVM to the number of hidden layers, neurons in a NN or dropout rate in a CNN. Hyperparameter optimisation is represented as - 

$$ x^*=arg\min_{x \in X } f(x)$$

$f(x)$ is the objective score to be minimised such as RMSE or error rate evaluated on the validation set; $x*$ is the set of hyperparameters that yields the lowest value of the score, and x can take on any value in the domain $X$.

The reason the objective function is a blackbox function is that that it is not always a strict mathematical function and the empirical optimum might not always the best. The hyperparameter search space can be extremely complex such as being mixed integer-continuous (eg. regularisation and kernel), often domain constrained (eg. positive regularisation), combinatorial (eg. feature selection) and also have conditional dimensions (eg. architecture of a NN). Therefore, a optimisation routine for hyperparameter search is ideally able to handle stochastic, non-convex and non-continuous functions, efficient in terms of function evaluations, flexible in terms of search space and parallelisable in execution. The main bottleneck is often evaluating the objective function $f(x)$.


This notebook aims to explore the different global black-box algorithms available for hyperparameter optimisation. There are four parts to an optimisation problem - 
1. Objective function: what we want to minimise
2. Domain space: values of the parameters over which to minimise the objective
3. Hyperparameter optimisation function: constructs the surrogate function and chooses next values to evaluate
4. Trials: score, parameter pairs recorded each time we evaluate the objective function


## Algorithms

### Grid Search

Grid search is has been the traditional way to perform hyperparameter optimisation which performs an exhaustive search through a manually specified subset of the hyperparamater search space to brute force all combinations (Fig 1). Grid search must be guided by some performance metric, either cross validation on the training set or evaluation on a held-out validation set. Grid search is easy to implemenet and parallelise. It is also common to perform multistage, multi-resolution grid experiments that are more or less automated, as the grid grows exponentially with the number of hyperparameters. Together with a low effective dimensionality, the grid is likely to be suboptimal since it will potentially cover many spaces of low importance while under-examining hyperparameters in areas of high importance.

<p></p>
<div style="width:image width px; font-size:80%; text-align:center;">
<img src="imgs/grid.png" width="200"/>Fig 1. Grid Search Layout</div>





### Random Search
Random search is a technique where values are sampled from a predefined distribution for each hyperparameter of interest to find the best solution for the built model (Fig 2). It is equally easy to implement and parallelise but has superior coomputational performance in higher-dimensional spaces. The drawback of random search is that it yields high variance during computing. Additionally, it can take a large number of samples to be reasonably confident that a solution close to the optimal has been found. With random search, it is also very difficult to gain confidence that a solution is close to the optimal solution, as there is never a visible convergence in the algorithm. 

<p></p>
<div style="width:image width px; font-size:80%; text-align:center;">
<img src="imgs/random.png" width="200"/>Fig 2. Random Search Layout</div>

***Papers***
- Random Search for Hyper-Parameter Optimization (2012) [Link](http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf)

#### True/Quantum Random Search

A computer, which is a completely deterministic machine you cannot generate a truly random numbers as it is following the same algorithm to generate them. Typically, that means it starts with a common ‘seed’ number and then follows a pattern. The results may be sufficiently complex to make the pattern difficult to identify, but because it is ruled by a carefully defined and consistently repeated algorithm, the numbers it produces are not truly random.

Therefore libraries such as Talos generate true random numbers either based on vacuum based quantum processes or ambient sound based randomness through an [API](https://github.com/autonomio/talos/issues/113) that produces these values mechanically.

***Links***
- [Paper] A generator for unique quantum random numbers based on vacuum states [Link](https://www.nature.com/articles/nphoton.2010.197) 
- [Paper] True Random Number Generation Based on Environmental Noise Measurements for Military Applications [Link](http://www.wseas.us/e-library/conferences/2009/cambridge/ISPRA/ISPRA09.pdf)
- [Article] Can a computer generate a truly random number? [Link](https://engineering.mit.edu/engage/ask-an-engineer/can-a-computer-generate-a-truly-random-number/)
- [Article] Audio Based True Random Number Generator POC [Link](https://www.guyrutenberg.com/2010/05/14/audio-based-true-random-number-generator-poc/)
- [Library] Talos [Link](https://github.com/autonomio/talos/blob/master/README.md)

#### Quasi-random Search

In the case if there are not enough data points, random sampling does not fully cover the parameter space. In addition, it samples some points very close to each other which can be redundant. We can solve this problem which low-discrepancy sequences (also called quasi-random sequences) to ensure sampled points are more equidistant from one another to ensure there are no "clumps" or "holes" from one another. One disadvantage of these methods is that not all of them provide good results in higher dimensions. For instance, Halton sequence and Hammersly set do not work well for dimensions larger than 10. 


<p></p>
<div style="width:image width px; font-size:80%; text-align:center;">
<img src="imgs/quasi.png" width="400"/>Quasi-random Search</div>

***Links***

- [Article] Poisson disc sampling in Python [Link](https://scipython.com/blog/poisson-disc-sampling-in-python/)
- [Paper] Controlled Random Search Improves Hyper-Parameter Optimization: Poisson Disk Sampling [Link](https://arxiv.org/pdf/1809.01712.pdf)
- [Paper] Fast Poisson Disk Sampling in Arbitrary Dimensions (2007) [Link](https://www.cs.ubc.ca/~rbridson/docs/bridson-siggraph07-poissondisk.pdf)
- [Library] bridson: 2D Poisson Disc Sampling using Robert Bridson's Algorithm [Link](https://github.com/emulbreh/bridson)
- [Library] Talos [Link](https://github.com/autonomio/talos/blob/master/README.md)

#### Hyperband

Hyperband randomly samples a set of hyperparameter configurations which are then trained for a certain amount of iterations before they are ranked based on their performance. The best performing configurations are then chosen and trained for an additional number of iterations. This process is repeated until only a few configurations remain which are then trained for the maximum number of iterations to find the best configuration.

***Links***
- [Paper] Hyperband: A Novel Bandit-Based Approach to Hyperparameter Optimization (2016) [Link](https://arxiv.org/abs/1603.06560)
- [Article] Hyperband: A Novel Bandit-Based Approach to Hyperparameter Optimization Demo (2016) [Link](https://people.eecs.berkeley.edu/~kjamieson/hyperband.html)
- [Article] Tuning hyperparams fast with Hyperband (2017) [Link](http://fastml.com/tuning-hyperparams-fast-with-hyperband/)
- [Library] hyperband [Link](https://github.com/zygmuntz/hyperband)

### Evolutionary Algorithm

An evolution algorithm is provides the user a set of candidate solutions to evaluate a problem. The evaluation is based on an objective function that takes a given solution and returns a single fitness value. Based on the fitness results of the current solutions, the algorithm will then produce the next generation of candidate solutions that is more likely to produce even better results than the current generation. The iterative process will stop once the best known solution is satisfactory for the user.

There are many forms of evolutionary algorithms, from Evolutionary Strategies (ES) to Genetic Algorithms (GA). Evolutionary Strategies use vectors of real valued numbers as the representation. GAs on the other hand are characterised by a binary string representation of candidate solutions.  It is not uncommon to combine with other methods such as Gaussian Processes. It can dynamically adapt its search resolution per hyperparameter, allowing for efficient searches on different scales.

#### Evolutionary Strategy

One of the simplest evolution strategy we can imagine will just sample a set of solutions from a Normal distribution, with a mean $\mu$ and a fixed standard deviation $\sigma$. After the fitness results are evaluated, we can set $\mu$ to the best solution in the population and sample the next generation of solutions around this new mean. This simple algorithm will generally only work for simple problems. Given its greedy nature, it throws away all but the best solution, and can be prone to be stuck at a local optimum for more complicated problems. It would be beneficial to sample the next generation from a probability distribution that represents a more diverse set of ideas, rather than just from the best solution from the current generation. The following shows a visualisation based on a 2D projection of the [Rastringin](https://en.wikipedia.org/wiki/Rastrigin_function) function.

<p></p>
<div style="width:image width px; font-size:80%; text-align:center;">
<img src="imgs/simplerases.gif" width="250"/>Evolutionary Strategy for Rastringin Function</div>

#### Genetic Algorithm
The basic idea is quite simple: keep only 10% of the best performing solutions in the current generation, and let the rest of the population die. In the next generation, to sample a new solution is to randomly select two solutions from the survivors of the previous generation, and recombine their parameters to form a new solution. This crossover recombination process uses a coin toss to determine which parent to take each parameter from. In the case of our 2D toy function, our new solution might inherit xx or yy from either parents with 50% chance. Gaussian noise with a fixed standard deviation will also be injected into each new solution after this recombination process. 

The visualisation below demonstrates how a simple GA works. The green dots represent members of the elite population from the previous generation, the blue dots are the offsprings to form the set of candidate solutions, and the red dot is the best solution. Genetic algorithms help diversity by keeping track of a diverse set of candidate solutions to reproduce the next generation. However, in practice, most of the solutions in the elite surviving population tend to converge to a local optimum over time. There are more sophisticated variations of GA out there, such as [CoSyNe](http://people.idsia.ch/~juergen/gomez08a.pdf), [ESP](http://blog.otoro.net/2015/03/10/esp-algorithm-for-double-pendulum/), and [NEAT](http://blog.otoro.net/2016/05/07/backprop-neat/), where the idea is to cluster similar solutions in the population together into different species, to maintain better diversity over time.

<p></p>
<div style="width:image width px; font-size:80%; text-align:center;">
<img src="imgs/simplerasga.gif" width="250"/>Genetic Algorithm for Rastringin Function</div>

####  Covariance Matrix Adaptation Evolution Strategy CMA-ES

A shortcoming of both the Simple ES and Simple GA is that our standard deviation noise parameter is fixed. There are times when we want to explore more and increase the standard deviation of our search space, and there are times when we are confident we are close to a good optima and just want to fine tune the solution. CMA-ES is an algorithm which adaptively updates the parameters of the search distribution for next generations. At each generation, CMA-ES provides the parameters of a multi-variate normal distribution to sample solutions from. 

CMA-ES modifies the covariance calculation formula in a clever way to make it adapt well to an optimisation problem. Firstly, it focuses on the $N_{best}$ solutions in the current generation. After sorting the solutions based on fitness, we calculate the mean $\mu^{(g+1)}$ of the next generation $(g+1)$ as the average of only the $N_{best}$ solutions in current population $(g)$. Next, we use only the $N_{best}$ of the solutions to estimate the covariance matrix $C^{(g+1)}$ of the next generation, but the clever hack here is that it uses the current generation’s $\mu^{(g)}$, rather than the updated $\mu^{(g+1)}$ parameters that we had just calculated. We then use this set of $\mu_{x},\mu_{y},\sigma_{x},\sigma_{y}$ and $\sigma_{xy}$ to sample for the next generation of candidate solutions.

<img src="imgs/cmaes.png" alt="drawing" width="700"/>

1. Calculate the fitness score of each candidate solution in generation $(g)$.
2. Isolate the $N_{best}$% of the population in then generation $(g)$ (purple).
3. Using only the best solutions, with the mean $\mu^{(g)}$ of the current generation (green dot), calculate the covariance matrix $C^{(g+1)}$ of the next generation.

$$
\displaystyle {\sigma_{xy}^{(g+1)} = \frac{1}{N_{best}} \sum_{i=1}^{N_{best}}(x_i - \mu_x^{(g)})(y_i - \mu_y^{(g)})}
$$

4. Sample a new set of candidate solutions using the updated mean $\mu^{(g+1)}$ and covariance matrix $C^{(g+1)}$.

<p></p>
<div style="width:image width px; font-size:80%; text-align:center;">
<img src="imgs/cmaes.gif" width="250"/>Visualisation of CMAES on Rastringin function</div>

***Links***
- [Article] Wikipedia:CMA-ES [Link](https://en.wikipedia.org/wiki/CMA-ES#Example_code_in_MATLAB/Octave)
- [Paper] CMA-ES for Hyperparameter Optimization of Deep Neural Networks (2016) [Link](https://arxiv.org/pdf/1604.07269.pdf)

#### Particle Swarm Optimisation PSO

Particle swarm optimisation aims to find the global optima in a search space using swarm of particles. These particles move in a direction guided by the particle's own previous velocity (inertia), distance from the particle's best known position (cognitive force) and distance from the swarms best known position (social force). Essentially the particles collectively communicate with each other to converge faster. The swarm does not fully explore the search space but potentially finds a better solution.

<div style="width:image width px; font-size:80%; text-align:center;">
<img src="imgs/psomvt.png" width="500" style="padding-bottom:0.5em;" />Particle movement is overpowered by the swarm direction </div>


Each iteration of a particle's velocity is updated by using - 

$$
v^{(i+1)} = w \cdot v^{(i)} + c_1 \cdot r_1 \cdot \bigg(p_{best}^{(i)} - p^{(i)}\bigg) + c_2 \cdot r_2 \cdot \bigg(p_{gbest}^{(i)}-p^{i}\bigg)
$$

$$
p^{(i+1)}=p^{(i)}+v^{(i)}
$$

The coefficient $w$ denotes the inertia weight, $c_1$ and $c_2$ are the acceleration coefficients called cognitive and social parameters respectively, due to their role in the evolution procedure. The three coefficients are employed to control the balance of an individual particle:s self-learning vs the learning rate of the PSO population. $r_1$ and $r_2$ are randomly generated values that introduce a stochastic component to the search algorithm to extend the search space covered.

<br></br>
<table><tr>
    <td><div style="width:image width px; font-size:80%; text-align:center;">
<img src="imgs/pso.gif" width="250" style="padding-bottom:0.5em;" /></div></td>
    <td><div style="width:image width px; font-size:80%; text-align:center;">
<img src="imgs/3dpso.gif" width="250" style="padding-bottom:0.5em;" />
        </div></td>
    </tr></table>

***Links***
- [Paper] Analysis of Particle Swarm Optimization Algorithm (2010) [Link](https://pdfs.semanticscholar.org/6d00/7fd5f734cb4c6c9457e22f46a58be5edb662.pdf)
- [Article] Wikipedia:PSO [Link](https://en.wikipedia.org/wiki/Particle_swarm_optimization)
- [Article] Hyperparameter Optimisation Utilising a Particle Swarm Approach (2018) [Link](https://medium.com/next-level-german-engineering/hyperparameter-optimisation-utilising-a-particle-swarm-approach-5711957b3f3f)
- [Library] SwarmOpt [Link](https://github.com/SioKCronin/swarmopt)


### Sequential Model-Based Optimisation SMBO/Bayesian Optimisation

One of the main problems of optimising hyperparameters for learning algorithms is that it can take a long time to evaluate a set of hyperparameters. As a result, sequential model-based optimisation (SMBO) methods have been employed in settings when the performance evaluation of a model is expensive. SMBO, a formalisation of Bayesian Optimisation, takes the approach of sequentially constructing probabilistic models (otherwise known as surrogate functions) $P(parameters|score)$ to approximate the performance of the hyperparameters based on historical measurements and subsequently chooses new hyperparameters to test based on this model to reduce the number of evaluations of the true objective function. Evaluating the surrogate function is much cheaper than evaluating the objective function itself and exploits uncertainty to balance exploration vs exploitation. The algorithm is therefore - 

1. Build a surrogate probability model of the objective function
2. Find the hyperparameters that perform best on the surrogate
3. Apply these hyperparameters to the true objective function
4. Update the surrogate model incorporating the new results
5. Repeat steps 2–4 until max iterations or time is reached

<p></p>
<div style="width:image width px; font-size:80%; text-align:center;">
<img src="imgs/bayesopt.gif" width="500" style="padding-bottom:0.5em;" /></div>


There are five main aspects of model-based optimisation - 
1. A domain of hyperparameters of which to search (defined probability distributions for each hyperparameter)
2. An objective function (machine learning model) which takes in hyperparameters and outputs score to maximise (or minimise)
3. The surrogate model (probability model) of the objective function
4. The acquisition function (criteria or selection function) for evaluating which hyperparameter to choose next from the surrogate model
5. A history consisting of [score, hyperparameter] pairs used by the algorithm to update the surrogate model


Most variants of SMBO methods differ in how they build the surrogate function and the acquisition function used to select the next hyperparameters. The surrogate function is sometimes also known as a response surface as it the probability representation of the response surface because it is a high-dimensional mapping of hyperparameters to the probability of a score on the objective function. Options for the surrogate function range from Gaussian process, Random Forest Regression to Tree-structured Parzen Estimators (TPE).

###### Expected Improvement
The acquisition function is the criteria by which the next set of hyperparameters are chosen from the surrogate function. The most common choice of criteria is Expected Improvement - 
$$
EI_{y^*}(x)=\int_{-\infty}^{y^*}({y^*}-y)p(y|x)dy
$$

Here $y^*$ is a threshold value of the objective function, $x$ is the proposed set of hyperparameters, $y$ is the actual value of the objective function using hyperparameters $x$, and $p(y|x)$ is the surrogate probability model expressing the probability of $y$ given $x$. The aim is to maximise $EI$ with respect to $x$ which means to find the best hyperparameters under the surrogate function $p(y|x)$. If $p(y|x)$ is zero everywhere that $y<y^*$, then the hyperparameters $x$ are not expected to yield any improvement. If the integral is positive, then the hyperparameters $x$ are expected to yield a better result than the threshold value.

There are other acquisition functions such as probability of improvement (PI) and upper confidence bound (UCB), however expected improvement (EI) is the most commonly used and handles uncertainty and noise well.

[This paper](https://arxiv.org/pdf/1807.02811.pdf), A Tutorial on Bayesian Optimization, speaks of novel acquisition functions such as knowledge-gradient which is an extension of EI and entropy search and predictive entropy search which values the information about hte locaiton global maximum according to differential entropy. There are also multi-step optimal acquisition functions available but studies show that they do not yet perform better than one-step acquistion functions. 


***Links***
- [Paper] Making a Science of Model Search: Hyperparameter Optimisation in Hundreds of Dimensions for Vision Architectures (2013) [Link](http://proceedings.mlr.press/v28/bergstra13.pdf)
- [Paper] Bayesian Optimisation Primer (2017) [Link](https://app.sigopt.com/static/pdf/SigOpt_Bayesian_Optimization_Primer.pdf)
- [Paper] Taking the Human Out of the Loop: A Review of Bayesian Optimisation [Link](https://www.cs.ox.ac.uk/people/nando.defreitas/publications/BayesOptLoop.pdf)
- [Paper] A Stratified Analysis of Bayesian Optimization Methods [Link](https://arxiv.org/pdf/1603.09441.pdf)
- [Paper] Fast Bayesian Optimization of Machine Learning Hyperparameters on Large Datasets [Link](https://arxiv.org/pdf/1605.07079.pdf)
- [Paper] mlrMBO: A Modular Framework for Model-Based Optimization of Expensive Black-Box Functions [Link](https://arxiv.org/pdf/1703.03373.pdf)
- [Article] Harvard Intelligent Probabilistic Systems: A Tutorial on Bayesian Optimization for Machine Learning [Link](https://www.iro.umontreal.ca/~bengioy/cifar/NCAP2014-summerschool/slides/Ryan_adams_140814_bayesopt_ncap.pdf)
- [Article] Expected Improvement for Bayesian Optimization: A Derivation [Link](http://ash-aldujaili.github.io/blog/2018/02/01/ei/)

#### Gaussian Processes GP

Gaussian Processes (GPs) provide a rich and flexible class of non-parametric statistical models over function spaces with domains that can be continuous, discrete, mixed, or even hierarchical in nature. Furthermore, the GP provides not just information about the likely value of f, but importantly also about the uncertainty around that value.

The idea behind Gaussian Process Regression is for a set of observed values $F_N$ at some points $X_N$, we assume that these values correspond to the realisation of a multivariate Gaussian Process with a prior distribution - 

$$
p(F_N|X_N) = \frac{exp(-\frac{1}{2}F_N^T K_N^{-1}F_N)}{\sqrt{(2\pi)^Ndet(K_N)}}
$$

where $K_N$ is a $N\times N$ covariance matrix and its coefficients are expressed in terms of a correlation function (or kernel) $Kmn=K(xm, xn, \theta)$. The hyperparameters $\theta$ of the kernel are calibrated according to a maximum likelihood principle. $K_N$ is chosen to reflect a prior assumption of the function and therefore the choice of the kernel will have a significant impact on the correctness of the regression. 

<p></p>
<div style="width:image width px; font-size:80%; text-align:center;">
<img src="imgs/kernel.png" width="500" style="padding-bottom:0.5em;" />Covariance Functions (a) and sampled data (B)</div>

Through mathematical transformations and using the conditional probability rule it is possible to estimate the posterior distribution $p(N+1|F_N, X_N +1)$ and express $f^{N+1}$ as a function of $K_N$ and $F_N$ with an uncertainty. This allows us to construct a probabilistic surrograme from our observations.

<p></p>
<div style="width:image width px; font-size:80%; text-align:center;">
<img src="imgs/bayesopt2d.gif" width="500" style="padding-bottom:0.5em;" /></div>


***Links***
- [Paper] Bayesian Gaussian Processes for Sequential Prediction, Optimisation and Quadrature [Link](https://pdfs.semanticscholar.org/74d6/b7a3fda8ba7e9183ccbb8b697018b38864d7.pdf)
- [Article] The intuitions behind Bayesian Optimization with Gaussian Processes [Link](https://towardsdatascience.com/the-intuitions-behind-bayesian-optimization-with-gaussian-processes-7e00fcc898a0)


#### Sequential model-based algorithm configuration SMAC

SMAC models the surrogate function as a random forest. When performing cross-validation, SMAC only evaluates as many folds as necessary to show that a configuration is worse than the best one seen so far. SMAC can handle continuous, categorical, and conditional parameters.

***Links***
- [Paper] Sequential Model-Based Optimization for General Algorithm Configuration [Link](https://ml.informatik.uni-freiburg.de/papers/11-LION5-SMAC.pdf)
- [Article] Dr. Frank Hutter:Optimization of Machine Learning Hyperparameters [Link](http://aad.informatik.uni-freiburg.de/~hutter/ML3.pdf)


#### Tree-structured Parzen Estimator TPE

The Tree-structured Parzen Estimator uses Bayes rule to build the surrogate model instead of directly applying $p(y|x)$.

$$
p(y|x)=\frac{p(x|y)p(y)}{p(x)}
$$

$p(x|y)$ which is the probability of the hyperparameters given the score on the objective function is expressed as - 

$$
   p(x|y) = 
    \begin{cases}
      l(x) & \text{if $y<y^*$} \\
      g(x) & \text{if $y \geq y^*$}
    \end{cases}
$$

where $y<y^*$ represents a lower value of the objective function than the threshold. This equation shows that there are two different distributions for the hyperparameters: one where the value of the objective function is less than the threshold, $l(x)$, and one where the value of the objective function is greater than the threshold, $g(x)$. 

<div style="width:image width px; font-size:80%; text-align:center;">
<img src="imgs/tpe.png" width="350" style="padding-bottom:0.5em;" />TPE Probability Distributions</div>

This distribution is built up using Parzen-window density estimation also known kernel density estimation hence Parzen in the name. Kernel density estimation is a non-parametric way to build a probability density function from empirical data where each sample defines a Gaussian distribution with mean equal to the hyperparameter value and some specified standard deviation. These Gaussian distributions are then stacked together and normalised to assure we get a valid probability distribution.

<div style="width:image width px; font-size:80%; text-align:center;">
<img src="imgs/parzen.png" width="450" style="padding-bottom:0.5em;" />Parzen density estimation for four samples</div>

Intuitively, we want to draw values of x from $l(x)$ and not from $g(x)$ because this distribution is based only on values of $x$ that yielded lower scores than the threshold. With Bayes Rule, and a few substitutions -  $\big(\gamma=p(y<y^*)\big)$ and $\big(p(x)=\int_{\mathbb{R}} p(x|y)dy=\gamma l(x)+(1-\gamma)g(x)\big)$ - the expected improvement equation (which we are trying to maximise) becomes - 

$$
EI_{y^*}(x)=\frac{\gamma y^*l(x)-l(x) \int_{-\infty}^{y^*}p(y)dy}{\gamma l(x)+ (1- \gamma)g(x)} \propto \bigg(\gamma + \frac{g(x)}{l(x)} (1-\gamma)\bigg)^{-1}
$$

The right term says the $EI$ is proportional to the ratio $\frac{l(x)}{g(x)}$ and therefore to maximimse $EI$, we need to maximise this ratio meaning we would like points $x$ with high probability under $l(x)$ and low probability under $g(x)$. TPE works by drawing sample hyperparameters from $l(x)$, evaluating them in terms of $\frac{l(x)}{g(x)}$, and returning the set that yields the highest value under $\frac{l(x)}{g(x)}$ corresponding to the greatest $EI$. These hyperparameters are then evaluated on the objective function which should yield a better value when evaluated. 

<p></p>
<div style="width:image width px; font-size:80%; text-align:center;">
<img src="imgs/tpe_ei.png" width="450" style="padding-bottom:0.5em;" />Calculation of a candidate with best Expected Improvement</div>


The expected improvement criteria allows the model to balance exploration vs exploitation. $l(x)$ is a distribution and not a single value which means that the hyperparameters drawn are likely close but not exactly at the maximum of the expected improvement. Moreover, because the surrogate is just an estimate of the objective function, the selected hyperparameters may not actually yield an improvement when evaluated and the surrogate model will have to be updated. This updating is done based on the current surrogate model and the history of objective function evaluations. Each time the algorithm proposes a new set of candidate hyperparameters, it evaluates them with the actual objective function and records the result in a pair (score, hyperparameters) which then forms the history. The algorithm builds $l(x)$ and $g(x)$ using the history to come up with a probability model of the objective function that improves with each iteration. This is Bayes’ Rule at work: we have an initial estimate for the surrogate of the objective function that we update as we gather more evidence. Eventually, with enough evaluations of the objective function, we hope that our model accurately reflects the objective function and the hyperparameters that yield the greatest Expected Improvement correspond to the hyperparameters that maximise the objective function.

***Links***
- [Paper] Algorithms for Hyper-Parameter Optimisation (2011) [Link](https://papers.nips.cc/paper/4443-algorithms-for-hyper-parameter-optimization.pdf)
- [Article] A Conceptual Explanation of Bayesian Hyperparameter Optimization for Machine Learning [Link](https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f)
- [Article] Automated Machine Learning Hyperparameter Tuning in Python [Link](https://towardsdatascience.com/automated-machine-learning-hyperparameter-tuning-in-python-dfda59b72f8a)
- [Article] Hyperparameter optimization: Explanation of automatized algorithms [Link](http://dkopczyk.quantee.co.uk/hyperparameter-optimization/)
- [Library] Hyperopt: Distributed Asynchronous Hyper-parameter Optimization [Link](https://github.com/hyperopt/hyperopt): [Examples](hyperopt.ipynb) 
- [Library] Hyperas: Wrapper for hyperopt [Link](https://github.com/maxpumperla/hyperas)


#### Radial Basis Function 

**Libraries**
- RBFOpt [Link](https://github.com/coin-or/rbfopt)

**Papers**
- An effective algorithm for hyperparameter optimization of neural networks (2017) [Link](https://arxiv.org/abs/1705.08520)
- Efficient Hyperparameter Optimization of Deep Learning Algorithms Using Deterministic RBF Surrogates (2016) [Link](https://arxiv.org/abs/1607.08316)

## Direct Search Methods

### Nelder Meads method

https://timvieira.github.io/blog/post/2018/03/16/black-box-optimization/


**Papers**
- Effective hyperparameter optimization using Nelder-Mead method in deep learning [Link](https://ipsjcva.springeropen.com/articles/10.1186/s41074-017-0030-7)





## Evaluation
The choice of the sampling/search strategy depends strongly on the problem tackled. Ultimately, their are 4 aspects of the problem to look at:

1. the time required to evaluate a model,
2. the number of variables,
3. the type of variable (continuous or discrete),
4. the conditionality of the search space.


- [Grid Search](#Grid-Search) applies when all variables are discrete and the number of possibilities is low. A grid search will perform the exhaustive combinatorial search over all possibilities making the search extremely long even for medium sized problems.


- [Random Search](#Random-Search) is an alternative to grid search when the number of discrete parameters to optimise and the time required for each evaluation is high. When all parameters are discrete, random search will perform sampling without replacement making it an algorithm of choice when combinatorial exploration is not possible. With continuous parameters, it is preferable to use quasi random sampling.


- [Quasi-Random](#Quasi-random-Search) sampling ensures a much more uniform exploration of the search space than traditional pseudo random. Thus, quasi random sampling is preferable when not all variables are discrete, the number of dimensions is high and the time required to evaluate a solution is high.

<p></p>
<div style="width:image width px; font-size:80%; text-align:center;">
<img src="imgs/random_result.png" width="400"/>Sobol Sequence sampling is slightly more efficient than Latin hypercube sampling and uniform random sampling</div>


- [Bayes search models](#Sequential-Model-Based-Optimisation-SMBO/Bayesian-Optimisation) the search space to give an estimate of the loss function and the uncertainty on that estimate at every point of the search space. Modelling the search space suffers from the curse of dimensionality, which makes this method more suitable when the number of dimensions is low. Moreover, since it models both the expected loss and uncertainty, this search algorithm converges in few steps on superior configurations, making it a good choice when the time to complete the evaluation of a parameter configuration is high.



- [CMA-ES Search](#Covariance-Matrix-Adaptation-Evolution-Strategy-CMA-ES) is one of the most powerful black-box optimisation algorithms. However, it requires a significant number of model evaluation (in the order of 10 to 50 times the number of dimensions) to converge to an optimal solution. This search method is more suitable when the time required for a model evaluation is relatively low.


- MOCMAES search is a multi-objective algorithm optimising multiple tradeoffs simultaneously. To do that, MOCMAES employs $\mu$ CMA-ES algorithms. Thus requiring even more evaluation to converge to the optimal solution (in the order of $\mu$  times 10 to 50, times the number of dimensions). This search method is more suitable when the time required for a model evaluation is relatively low.


<p></p>
<div style="width:image width px; font-size:80%; text-align:center;">
<img src="imgs/eval1.png" width="450" style="padding-bottom:0.5em;" />Evaluation of Random Search, TPE and PSO on 2D Rastringin Function</div>

While the Bayesian Methods perhaps consistently outperform random sampling, they do so only by a negligible amount. To quantify this idea, we compare to random run at twice the speed which beats the two Bayesian Optimization methods, i.e., running random search for twice as long yields superior results.

<p></p>
<div style="width:image width px; font-size:80%; text-align:center;">
<img src="imgs/hyperband.png" width="250" style="padding-bottom:0.5em;" /></div>